In [1]:
import spotiphy
import numpy as np
import matplotlib as mpl
import scanpy as sc
import squidpy as sq
import importlib as imp
imp.reload(spotiphy)
imp.reload(spotiphy.segmentation)
imp.reload(spotiphy.sc_reference)

<module 'spotiphy.sc_reference' from 'C:\\Users\\zzheng92\\Desktop\\Spotiphy\\spotiphy\\sc_reference.py'>

In [2]:
adata_sc = sc.read_h5ad("F:/Ziqian Zheng/Spatial omics/Public dataset/scRNA/scRNA.h5ad")
# adata_st = sc.read_visium("F:/Ziqian Zheng/Spotiphy_data/ST/Sample221/outs")
adata_st = sc.read_h5ad("F:/Ziqian Zheng/Spotiphy_data/Simulated ST/Simulated_ST_221_1.h5ad")
adata_st.var_names_make_unique()
img = sq.im.ImageContainer('F:/Ziqian Zheng/Spotiphy_data/ST/Sample221/Sample221.jpg')
results_folder = 'results/simulation1/'
key_type = 'Allen.subclass_label'
type_list = sorted(list(adata_sc.obs[key_type].unique()))

Image.py (3176): Image size (99010000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.


In [3]:
%%time
Segmentation = spotiphy.segmentation.Segmentation(img['image'][:, :, 0], adata_st.obsm['spatial'], out_dir=results_folder)
Segmentation.segment_nucleus(save=True)
n_cell_df = Segmentation.n_cell_df

Suppress the output of tensorflow prediction for tensorflow version 2.12.0>=2.9.0.
Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


100%|██████████| 36/36 [01:07<00:00,  1.89s/it]


CPU times: total: 22min 19s
Wall time: 1min 30s


In [4]:
%%time
adata_sc, adata_st = spotiphy.initialization(adata_sc, adata_st)

CPU times: total: 2min 36s
Wall time: 2min 36s


In [5]:
%%time
marker_gene = spotiphy.marker_selection(adata_sc, key_type=key_type, return_dict=False, n_select=40)
adata_sc = adata_sc[:, marker_gene]
adata_st = adata_st[:, marker_gene]

CPU times: total: 1min 1s
Wall time: 1min 1s


In [6]:
%%time
sc_ref = spotiphy.construct_sc_ref(adata_sc, key_type=key_type)
# spotiphy.plot_sc_ref(sc_ref, type_list)

CPU times: total: 1min 25s
Wall time: 1min 25s


In [7]:
%%time
X = np.array(adata_st.X)
pyro_params = spotiphy.deconvolution.deconvolute(X, sc_ref)
sigma = pyro_params['sigma'].cpu().detach().numpy()
mean_exp = np.array([np.mean(np.sum(adata_sc.X[adata_sc.obs[key_type]==type_list[i]], axis=1)) for i in range(len(type_list))])
cell_proportion = sigma/mean_exp
cell_proportion = cell_proportion/np.sum(cell_proportion, axis=1)[:, np.newaxis]
adata_st.obs[type_list] = cell_proportion
np.save(results_folder+'proportion.npy', cell_proportion)

100%|██████████| 8000/8000 [03:57<00:00, 33.64it/s]


CPU times: total: 5min 27s
Wall time: 5min 24s


<timed exec> (7): Trying to modify attribute `.obs` of view, initializing view as actual.
